<a href="https://colab.research.google.com/github/truongthuanr/self-project/blob/dev/11_Named_Entity_Recognition/Named_Entity_Recognition.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:

%%capture
!pip3 install datasets


In [2]:

import numpy as np
import pandas as pd
import tensorflow as tf
from keras.preprocessing.sequence import pad_sequences
from datasets import load_dataset
import os


In [3]:
from keras.models import Model
from keras.layers import Input,Embedding,TimeDistributed,\
                         Dropout,Conv1D,MaxPooling1D,\
                         Flatten,Bidirectional,LSTM,Dense,\
                         concatenate

from keras.initializers import RandomUniform

from keras.optimizers import Adam





In [4]:
# For vetorization
max_word_tokens = 24000
max_sentence_length = 30
max_word_len = 20


# Dataset

Used dataset:

https://huggingface.co/datasets/conll2003

In [5]:
conll_data = load_dataset("conll2003")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

# Data preprocessing


## Character processing

In [6]:
char2Idx = {"PADDING":0, "UNKNOWN":1}
for c in " 0123456789abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ.,-_()[]{}!?:;#'\"/\\%$`&=*+@^~|":
    char2Idx[c] = len(char2Idx)
len(char2Idx)

95

In [7]:
def character_vectorize(X):
  data_vec = []
  for sentence in X:
    # print(f"Sentence: {sentence}")
    padchar = char2Idx['PADDING']
    sentence_vec=[]
    for word in sentence:
      # print(word)
      chars = []
      if len(word) >= max_word_len:

        chars=[char2Idx[c] for c in word[:max_word_len]]
      else:
        prepad=int((max_word_len-len(word))/2)
        postpad=max_word_len-(len(word)+prepad)
        chars.extend([padchar]*prepad)
        chars.extend([char2Idx[c] for c in word])
        chars.extend([padchar]*postpad)
      sentence_vec.append(chars)

    data_vec.append(sentence_vec)
  data_vec = np.asarray(data_vec, dtype=object)
  return data_vec




# X_train_char = conll_data['train']['tokens']

# X_train_char = character_vectorize(X_train_char)
# X_train_char = pad_sequences(sequences = X_train_char,
#                              maxlen=max_sentence_length,
#                              dtype=object,
#                              padding="post",
#                              truncating="post",
#                              value=0)
# X_train_char = np.asarray(X_train_char,
#                           dtype=np.float32)

In [8]:
def char_preprocessing(char_input):
  '''
  input:: char_input: list of words
  output: vecterized array of character
  with shape(sentences, max_sentence_len, max_word_len)
  '''
  X_chars = character_vectorize(char_input)
  X_chars = pad_sequences(sequences = X_chars,
                          maxlen=max_sentence_length,
                          dtype=object,
                          padding="post",
                          truncating="post",
                          value=0)

  X_chars = np.asarray(X_chars,
                       dtype=np.float32)

  return X_chars

In [56]:
X_train_char = conll_data['train']['tokens']
X_train_char = char_preprocessing(X_train_char)

In [10]:
X_train_char.shape

(14041, 30, 20)

## Word preprocessing

In [11]:
# X_train_word = conll_data['train']['tokens']
X_train_word = np.array([ " ".join(row) for row in conll_data['train']['tokens']])
# X_train_word

## Caps preprocessing

In [53]:
caps2id = {'allcaps':0,
           'upperinitial':1,
           'lower':2,
           'mixedcaps':3,
           'noinfo':4}
id2caps = {v:k for k,v in caps2id.items()}

In [74]:
def caps_vectorize(input):
  '''
  input: array of sentences, sentencs is list of word
  '''
  cap_vec = []
  for sentence in input:
    sen_cap_type = []
    for word in sentence:
      # if word is Title
      if word.istitle():
        sen_cap_type.append(caps2id['upperinitial'])

      # if uper, lower, mixed or else
      else:
        if word.isupper():
          sen_cap_type.append(caps2id['allcaps'])
        elif word.islower():
          sen_cap_type.append(caps2id['lower'])
        else:
          if word.lower().islower():
            sen_cap_type.append(caps2id['mixedcaps'])
          else: sen_cap_type.append(caps2id['noinfo'])
    cap_vec.append(sen_cap_type)
  return cap_vec



In [75]:
'asfaDFAFdsafas'.lower()

'asfadfafdsafas'

In [77]:
def caps_preprocesing(X_input):
  X_caps = caps_vectorize(X_input)
  X_caps = pad_sequences(sequences = X_caps,
                        maxlen=max_sentence_length,
                        dtype=object,
                        padding="post",
                        truncating="post",
                        value=caps2id['noinfo'])

  return X_caps

X_train_caps = caps_preprocesing(conll_data['train']['tokens'])

In [81]:
X_train_caps[0]

array([0, 2, 1, 2, 2, 2, 1, 2, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4,
       4, 4, 4, 4, 4, 4, 4, 4], dtype=object)

## Targer preprocessing

In [12]:
label2id = {'O': 0, 'B-PER': 1, 'I-PER': 2, 'B-ORG': 3, 'I-ORG': 4, 'B-LOC': 5, 'I-LOC': 6, 'B-MISC': 7, 'I-MISC': 8}
id2label = {v: k for k, v in label2id.items()}


In [13]:
y_train = pad_sequences(conll_data['train']['ner_tags'],
                        dtype='int32',
                        maxlen=max_sentence_length,
                        padding='post',
                        truncating='post',
                        value=0,)

In [14]:
def encoding_ytrain(y_train):
  n=len(label2id)
  nrow=len(y_train)
  encoded = np.zeros(shape=(nrow,max_sentence_length,n))
  for iy, y in enumerate(y_train):
    senvec = np.zeros(shape=(max_sentence_length,n))
    for ichar, char in enumerate(y):
      charvec = np.zeros(n)
      charvec[char] = 1
      # print(f"{char=}")
      # print(f"{charvec=}")
      senvec[ichar]=charvec

    encoded[iy] = senvec
  return encoded

y_train = encoding_ytrain(y_train)


# Model

In [15]:
from keras.layers import TextVectorization

In [16]:
text_vectorizer = TextVectorization(max_tokens=max_word_tokens,   # how many word in the vocab lib
                                    standardize='lower_and_strip_punctuation', # process the tab
                                    split='whitespace', # split the token
                                    ngrams=None,
                                    output_mode='int', # map token to number
                                    output_sequence_length=max_sentence_length,
                                    pad_to_max_tokens=False,
                                    vocabulary=None,)
text_vectorizer.adapt(np.array([ " ".join(row) for row in conll_data['train']['tokens']]))

In [17]:
# Word level
word_input = Input(shape=(1,),dtype=tf.string,name='word_input')
# Tokenize

words_vectorize = text_vectorizer(word_input)

# Embedding
embedding = Embedding(input_dim=max_word_tokens,
                      output_dim=128,
                      embeddings_initializer="uniform",
                      input_length=None)
words = embedding(words_vectorize)


In [19]:
# Character level
character_input=Input(shape=(None,max_word_len,),
                      name='char_input',
                      dtype=tf.int32)
embed_char_out=TimeDistributed(Embedding(input_dim = len(char2Idx),
                                         output_dim=10,
                                         embeddings_initializer=RandomUniform(minval=-0.5, maxval=0.5),
                                         input_length=max_sentence_length,),
                                         name='char_embedding')(character_input)
dropout = Dropout(0.5)(embed_char_out)
conv1d_out= TimeDistributed(Conv1D(kernel_size=3,
                                   filters=10,
                                   padding='same',activation='tanh', strides=1))(dropout)
maxpool_out=TimeDistributed(MaxPooling1D(max_word_len))(conv1d_out)
char = TimeDistributed(Flatten())(maxpool_out)
char = Dropout(0.5)(char)

In [20]:
print(character_input.shape)
print(embed_char_out.shape)
print(dropout.shape)
print(conv1d_out.shape)
print(maxpool_out.shape)
print(char.shape)

(None, None, 20)
(None, None, 20, 10)
(None, None, 20, 10)
(None, None, 20, 10)
(None, None, 1, 10)
(None, None, 10)


In [21]:

combined = concatenate([words, char])
# combined = words
# Bi-LSTM
output = Bidirectional(LSTM(200, return_sequences=True, dropout=0.50, recurrent_dropout=0.25))(combined)

# # Output -> Tag Scoring -> Tag
output = TimeDistributed(Dense(len(label2id), activation='softmax'))(output)
model = Model(inputs=[word_input, character_input], outputs=[output])


In [22]:
model.compile(loss="binary_crossentropy",
              optimizer=Adam(),
              metrics=["accuracy"])

In [23]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 char_input (InputLayer)     [(None, None, 20)]           0         []                            
                                                                                                  
 char_embedding (TimeDistri  (None, None, 20, 10)         950       ['char_input[0][0]']          
 buted)                                                                                           
                                                                                                  
 dropout (Dropout)           (None, None, 20, 10)         0         ['char_embedding[0][0]']      
                                                                                                  
 time_distributed (TimeDist  (None, None, 20, 10)         310       ['dropout[0][0]']         

In [24]:
model.fit([X_train_word[:1000],X_train_char[:1000]],
          y_train[:1000],
          epochs=2,
          validation_data=([X_train_word[1000:1100],X_train_char[1000:1100]], y_train[1000:1100]))

Epoch 1/2
32/32 [==============================] - 23s 410ms/step - loss: 0.1107 - accuracy: 0.8803 - val_loss: 0.0627 - val_accuracy: 0.9443
Epoch 2/2
32/32 [==============================] - 12s 382ms/step - loss: 0.0717 - accuracy: 0.9083 - val_loss: 0.0576 - val_accuracy: 0.9443


# Predictions

In [25]:
X_train_word[0:2].shape

(2,)

In [29]:
def make_prediction(input_string):
  X_words = np.asarray([input_string])
  X_chars = [input_string.split()]
  X_chars = char_preprocessing(X_chars)
  predictions = model.predict([X_words,X_chars])


  # predictions = np.argmax()

  return predictions

In [46]:
predictions = make_prediction("I go to USA University by Germany tomorrow with Jane !")

1/1 [==============================] - 0s 42ms/step


In [49]:
predictions = make_prediction("Germany 's representative to the European Union 's veterinary committee Werner Zwingmann said on Wednesday consumers should buy sheepmeat from countries other than Britain until the scientific advice was clearer .")


1/1 [==============================] - 0s 43ms/step


In [45]:
X_train_word[4]

"Germany 's representative to the European Union 's veterinary committee Werner Zwingmann said on Wednesday consumers should buy sheepmeat from countries other than Britain until the scientific advice was clearer ."

In [48]:
y_train[4]

array([[0., 0., 0., 0., 0., 1., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 1., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 1., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 1., 0., 0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0., 0., 0., 0.],
       [1.

In [50]:
np.argmax(predictions,axis=2)

array([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0]])

# References

https://keras.io/examples/nlp/ner_transformers/